In [0]:
%pip install sparknlp mlflow

In [0]:
%pip install nltk

Q1: Word Count for Named Entities 

In [0]:
import com.johnsnowlabs.nlp

In [0]:
# Import stopwords-removing pipeline
from sparknlp.base import DocumentAssembler, Pipeline
from sparknlp.annotator import (
    SentenceDetector,
    Tokenizer,
    StopWordsCleaner
)
import pyspark.sql.functions as F
from sparknlp.pretrained import PretrainedPipeline

In [0]:
# Read in text file as data frame where each row corresponds to a line of the text file
# Link to file: https://www.gutenberg.org/cache/epub/345/pg345.txt
df = spark.read.option("header","false").option("inferSchema","true").option("delimiter","\n").csv("dbfs:/FileStore/tables/pg345.txt").toDF("text")

In [0]:
ner_pipeline = PretrainedPipeline("onto_recognize_entities_sm", lang = "en")


In [0]:
results = ner_pipeline.transform(df)

In [0]:
results.show()

In [0]:
results.select("entities.result").show(100, truncate=False)

In [0]:
# Extract just the named entities and save as RDD
entities = results.select(F.explode(results.entities.result).alias("entity")).rdd

In [0]:
# Run MapReduce on entities RDD to count number of named entities
# Finally...

entitiesKV = entities.map(lambda x : (x, 1))

In [0]:
entitiesKV.reduceByKey(lambda x, y : x + y).sortBy(lambda x : -x[1]).collect()